In [5]:
import os
import fitz
import pytesseract
from pdf2image import convert_from_path
from img2table.ocr import TesseractOCR
from img2table.document import Image
from img2table.document import pdf
from tabulate import tabulate
import json
import pandas as pd
import re
import time

In [2]:
def table_to_json_based_xlsx(xlsx_path, titles):
    xlsx_tables = []
    xlsx_file = pd.ExcelFile(xlsx_path)  # Replace with your file path
    sheet_names = xlsx_file.sheet_names

    for sheet_name in sheet_names:
        df = pd.read_excel(xlsx_file, sheet_name=sheet_name)

        match = re.search(r"Page (\d+) - Table (\d+)", sheet_name)
        if match:
            page_number, table_number = map(int, match.groups())
        else:
            page_number=''
            table_number=''

        if page_number!='' and table_number!='':
            title=[k[1] for k in titles if k[0]==(page_number, table_number)][0]

        else:
            title=''

        if df.empty:
            temp = [{}]
            for k in range(len(df.columns)):
                try:
                    temp[0][df.columns[k]] = ''
                except:
                    continue

            xlsx_tables.append({'info': (page_number, table_number), 'title': title, 'data': json.dumps(temp)})
        else:
            xlsx_tables.append({'info': (page_number, table_number), 'title': title, 'data': df.to_json(orient="records")})

    return xlsx_tables


def table_extract(source):
    return source.extract_tables(ocr=ocr,
                                 implicit_rows=True,
                                 borderless_tables=True,
                                 min_confidence=50)


def table_titles(tables_dict):
    titles = []
    for t in tables_dict:
        if tables_dict[t] != []:
            t_index=1
            for table in tables_dict[t]:
                titles.append([(t+1, t_index),table.title])
                t_index+=1

    return titles


def needs_ocr(pdf_path):
    doc = fitz.open(pdf_path)
    for page in doc:
        # Extract text using PyMuPDF
        text = page.get_text()
        if not text:
            return True  # PDF contains no searchable text
    return False


def find_text(page_text, word):
    start_index = 0
    indexes=[]
    while True:
        start_index = page_text.find(word, start_index)
        if start_index == -1:
            break

        end_index = start_index + len(word)

        indexes.append((start_index, end_index))
        start_index = end_index

    return indexes

def tabulate_converter(json_data):
    # Convert JSON to a list of dictionaries
    if type(json_data) == list:
        table_data = [list(record.values()) for record in json_data]
        # Get headers from the first record
        headers = list(json_data[0].keys())

    elif type(json_data) == dict:
        table_data = list(json_data.values())

        # Get headers from the first record
        headers = list(json_data.keys())

    # Convert to Markdown table
    markdown_table = tabulate(table_data, headers, tablefmt="pipe")

    return markdown_table

def extract_data(filepath):
    pdf_ = 0
    img_ = 0
    excel_ = 0

    if filepath.endswith((".pdf")):
        pdf_ = 1

        pdf = PDF(filepath,
                  detect_rotation=False,
                  pdf_text_extraction=True)

        pdf.to_xlsx(xlsx_path,
                    ocr=ocr,
                    implicit_rows=True,
                    borderless_tables=True,
                    min_confidence=90)

        extracted_tables = table_extract(pdf)
        titles = table_titles(extracted_tables)
        
        tables=table_to_json_based_xlsx(xlsx_path, titles)

        text_all = ''
        ocr_needed=0
        if needs_ocr(filepath):
            doc = convert_from_path(filepath, 500)
            ocr_needed=1
        else:
            doc = fitz.open(filepath)

        for i in range(len(doc)):
            page=doc[i]
            # Process each page
            if ocr_needed:
                text = pytesseract.image_to_string(page)
            else:
                text = page.get_text()  # Get all text on the page

            page_tables=[]
            for tb in tables:
                if tb['info'][0]==i+1:
                    page_tables.append(tb)

                if page_tables==[]:
                    text_all += text

                else:
                    for tbl in page_tables:
                        starting_words=[]
                        ending_words=[]
                        table_json=json.loads(tbl['data'])

                        if len(table_json)>=2:
                            starting_words.extend(table_json[0].keys())
                            starting_words.extend(table_json[0].values())

                            ending_words.extend(list(table_json[-1].values())[::-1])
                            ending_words.extend(list(table_json[-2].values())[::-1])
                        else:
                            starting_words.extend(table_json[0].keys())
                            starting_words.extend(table_json[0].values())

                            ending_words.extend(list(table_json[-1].values())[::-1])
                        
                        start_crop=0
                        end_crop=0
                        for w in starting_words:
                            if w!=None:
                                matches=find_text(text, str(w))
                                if len(matches)==1:
                                    start_crop=matches[0][0]
                                    break
                                else:
                                    continue
                            else:
                                continue

                        for w in ending_words:
                            if w!=None:
                                matches=find_text(text, str(w))
                                if len(matches)==1:
                                    end_crop=matches[0][1]
                                    break
                                else:
                                    continue
                            else:
                                continue

                        new_text=text[:start_crop]+'\n\n'+tabulate_converter(table_json)+'\n\n'+text[end_crop:]

                        new_text=new_text.split('| About')[0]
                        new_text=new_text.split('| f')[0]
                        new_text=new_text.split('| Contact Us')[0]

                        text=new_text
                        
                    text_all+=new_text

            for tbl in page_tables:
                tables.remove(tbl)

                
    return text_all

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
ocr = TesseractOCR(n_threads=1, lang="eng")
xlsx_path = 'temp/tables.xlsx'

source_dir = 'CBSL-data-p4'
output_dir = 'Extracted-text-CBSL-data'
count = 0
success_paths=[]

with open('errored-files.txt', 'r', encoding='utf-8') as error_file:
    error_filenames=error_file.read().split('\n')

for root, directories, files in os.walk(source_dir):
    for directory in directories:
        # Create corresponding directories in the output directory
        output_subdir = os.path.join(
            output_dir, os.path.relpath(root, source_dir), directory)
        os.makedirs(output_subdir, exist_ok=True)
        
    for file in files:
        if file.endswith('.pdf'):
            try:
                input_file_path = os.path.join(root, file)

                # Determine the output file path
                output_file_path = os.path.join(
                    output_dir, os.path.relpath(root, source_dir), file[:-3] + 'txt')

                if not os.path.exists(output_file_path) and input_file_path not in error_filenames:
                    text = extract_data(input_file_path)

                    # Write the extracted text to the output file
                    with open(output_file_path, 'w', encoding='utf-8') as write_file:
                        write_file.write(text)

                    print(f'{file} is extracted and saved to {output_file_path}')
                    count += 1
                    success_paths.append(input_file_path)

            except Exception as e:
                print(f'{e} - {input_file_path}')
                with open('errored-files.txt', 'a', encoding='utf-8') as error_file_write:
                    error_file_write.write(input_file_path+'\n')

for root, dirs, files in os.walk(output_dir, topdown=False):
    for dir in dirs:
        folder_path = os.path.join(root, dir)
        if not os.listdir(folder_path):  # Check if folder is empty
            os.rmdir(folder_path)
            print(f"Deleted empty folder: {folder_path}")

print(count)

Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl. - CBSL-data-p4\PUBLICATIONS\Economic and Financial Reports\Annual Report\Annual Report 1950\Money Supply.pdf


In [5]:
with open('count.txt', 'w', encoding='utf-8') as write_file:
    write_file.write(str(count))

21

In [ ]:
with open('file-paths.txt', 'w', encoding='utf-8') as write_file:
    for pt in success_paths:
        write_file.write(pt+'\n')

In [35]:
import fitz  # PyMuPDF

def pdf_page_to_json(pdf_path, page_number):
    # Open the PDF file
    doc = fitz.open(pdf_path)
    
    # Get the specified page
    page = doc.load_page(page_number - 1)
    
    # Extract the page text
    print(type(page))
    text = page.get_text('json')

    print(page.get_contents())

    # print(text)
    
    # Close the PDF
    # doc.close()
    
    # # Convert text to JSON format
    # json_data = {'page_number': page_number, 'text': text}
    
    # return json_data

# Example usage:
pdf_path = 'test2.pdf'  # Path to your PDF file
page_number = 1  # Page number to extract
pdf_page_to_json(pdf_path, page_number)
# print(json_data)


<class 'fitz.Page'>
{
 "width":595.3200073242188,
 "height":841.9199829101562,
 "blocks":[
  {
   "number":0,
   "type":0,
   "bbox":[
    72.02400207519531,
    71.9538345336914,
    93.22650146484375,
    85.4303970336914
   ],
   "lines":[
    {
     "spans":[
      {
       "size":11.039999961853027,
       "flags":4,
       "font":"Aptos",
       "color":0,
       "ascender":0.93896484375,
       "descender":-0.28173828125,
       "text":"Test ",
       "origin":[
        72.02400207519531,
        82.32000732421875
       ],
       "bbox":[
        72.02400207519531,
        71.9538345336914,
        93.22650146484375,
        85.4303970336914
       ]
      }
     ],
     "wmode":0,
     "dir":[
      1.0,
      0.0
     ],
     "bbox":[
      72.02400207519531,
      71.9538345336914,
      93.22650146484375,
      85.4303970336914
     ]
    }
   ]
  },
  {
   "number":1,
   "type":0,
   "bbox":[
    72.02400207519531,
    94.4137954711914,
    115.18250274658203,
    107.8903

In [28]:
from io import BytesIO
from img2table.document import PDF

pdf_path = "Test2.pdf"

pdf_from_path = PDF(src=pdf_path, pages=[0, 1])

# Definition of PDF from bytes
with open(pdf_path, 'rb') as f:
    pdf_bytes = f.read()
pdf_from_bytes = PDF(src=pdf_bytes)

# print(pdf_from_bytes)

pdf_from_file_like = PDF(src=BytesIO(pdf_bytes))

# print("="*100)
print(pdf_from_file_like)

extracted_tables  = pdf_from_file_like.extract_tables()

print(extracted_tables)
print("="*100)

PDF(src=<_io.BytesIO object at 0x0000028DE3B48310>, pages=None, detect_rotation=False, pdf_text_extraction=True, _rotated=False, _images=None)
{0: [ExtractedTable(title=Kjlhsdhaslkjhkjdshfgjkbsvjbasjcbcbaknkscbkjbadckjjbasdckjbscjlbakcjbkkjascbkadbckjjbsackj
bsckjbassckjbasckjbasckkjbasckjbdackjjba
sckjjbckajjsbckjabsckjbdackjbadcbadkjbckajbdckbeakjbcawkjcjbassjjccbakjsdbckjabcskjjeab
khbasckjbasckjjbascljbascljcbasljcbaljsjcbljasbclajsbclajsbcljasbcaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaacabsk, bbox=(184, 596, 842, 751),shape=(4, 2))]}


In [11]:
mat = fitz.Matrix(200 / 72, 200 / 72)
print("======================",mat)

doc = fitz.open(pdf_path)
    
    # Get the specified page
page = doc.load_page(page_number - 1)
pix = page.get_pixmap(matrix=mat)
print(pix)

====================== Matrix(2.7777777777777777, 0.0, 0.0, 2.7777777777777777, 0.0, 0.0)
Pixmap(DeviceRGB, (0, 0, 1654, 2339), 0)


In [24]:
def json_to_txt(json_data, output_file):
    with open(output_file, 'w') as f:
        for block in json_data['blocks']:
            for line in block['lines']:
                for span in line['spans']:
                    x1, y1, x2, y2 = span['bbox']
                    width = x2 - x1
                    height = y2 - y1
                    # Calculate text position based on bbox coordinates
                    text_x = x1
                    text_y = y1 + height / 2  # Considering vertical center alignment
                    f.write(f"{text_x}, {text_y}, {span['text']}\n")  # Write text with position to file



tx=tabulate_converter({"name":["suren","sss"],"age":[12,34]})
json={
    "width":595.3200073242188,
    "height":841.9199829101562,
    "blocks":[
     {
      "number":0,
      "type":0,
      "bbox":[
       72.02400207519531,
       71.9538345336914,
       93.22650146484375,
       85.4303970336914
      ],
      "lines":[
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":tabulate_converter({"name":["suren","sss"],"age":[12,34]}),
          "origin":[
           72.02400207519531,
           82.32000732421875
          ],
          "bbox":[
           72.02400207519531,
           71.9538345336914,
           93.22650146484375,
           85.4303970336914
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         72.02400207519531,
         71.9538345336914,
         93.22650146484375,
         85.4303970336914
        ]
       }
      ]
     },
     {
      "number":1,
      "type":0,
      "bbox":[
       72.02400207519531,
       94.4137954711914,
       115.18250274658203,
       107.8903579711914
      ],
      "lines":[
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"My Topic ",
          "origin":[
           72.02400207519531,
           104.77996826171875
          ],
          "bbox":[
           72.02400207519531,
           94.4137954711914,
           115.18250274658203,
           107.8903579711914
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         72.02400207519531,
         94.4137954711914,
         115.18250274658203,
         107.8903579711914
        ]
       }
      ]
     },
     {
      "number":2,
      "type":0,
      "bbox":[
       72.02400207519531,
       116.97379302978516,
       120.10250091552734,
       130.4503631591797
      ],
      "lines":[
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"Sub Topic ",
          "origin":[
           72.02400207519531,
           127.3399658203125
          ],
          "bbox":[
           72.02400207519531,
           116.97379302978516,
           120.10250091552734,
           130.4503631591797
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         72.02400207519531,
         116.97379302978516,
         120.10250091552734,
         130.4503631591797
        ]
       }
      ]
     },
     {
      "number":3,
      "type":0,
      "bbox":[
       72.02400207519531,
       139.41380310058594,
       521.9369506835938,
       196.4503631591797
      ],
      "lines":[
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"Kjlhsdhaslkjhkjdshfgjkbsvjbasjcbcbaknkscbkjbadckjjbasdckjbscjlbakcjbkkjascbkadbckjjbsackj",
          "origin":[
           72.02400207519531,
           149.77996826171875
          ],
          "bbox":[
           72.02400207519531,
           139.41380310058594,
           521.9369506835938,
           152.89036560058594
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         72.02400207519531,
         139.41380310058594,
         521.9369506835938,
         152.89036560058594
        ]
       },
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"bsckjbassckjbasckjbasckkjbasckjbdackjjba ",
          "origin":[
           72.02400207519531,
           164.29998779296875
          ],
          "bbox":[
           72.02400207519531,
           153.93382263183594,
           281.2001953125,
           167.41038513183594
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         72.02400207519531,
         153.93382263183594,
         281.2001953125,
         167.41038513183594
        ]
       },
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"sckjjbckajjsbckjabsckjbdackjbadcbadkjbckajbdckbeakjbcawkjcjbassjjccbakjsdbckjabcskjjeab",
          "origin":[
           72.02400207519531,
           178.82000732421875
          ],
          "bbox":[
           72.02400207519531,
           168.45384216308594,
           519.6962890625,
           181.93040466308594
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         72.02400207519531,
         168.45384216308594,
         519.6962890625,
         181.93040466308594
        ]
       },
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"khbasckjbasckjjbascljbascljcbasljcbaljsjcbljasbclajsbclajsbcljasbcaaaaaaaaaaaaaaaaaaaaaa",
          "origin":[
           72.02400207519531,
           193.3399658203125
          ],
          "bbox":[
           72.02400207519531,
           182.9738006591797,
           520.2195434570312,
           196.4503631591797
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         72.02400207519531,
         182.9738006591797,
         520.2195434570312,
         196.4503631591797
        ]
       }
      ]
     },
     {
      "number":4,
      "type":0,
      "bbox":[
       312.7900085449219,
       197.3738250732422,
       472.1424865722656,
       210.8503875732422
      ],
      "lines":[
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"aaaaaaaaaaaaaaaaaaaaaacabsk ",
          "origin":[
           312.7900085449219,
           207.739990234375
          ],
          "bbox":[
           312.7900085449219,
           197.3738250732422,
           472.1424865722656,
           210.8503875732422
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         312.7900085449219,
         197.3738250732422,
         472.1424865722656,
         210.8503875732422
        ]
       }
      ]
     },
     {
      "number":5,
      "type":0,
      "bbox":[
       312.7900085449219,
       219.93382263183594,
       559.2931518554688,
       247.81040954589844
      ],
      "lines":[
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"Dsdassaafsafasaaaaaaaaaaaa                                             ",
          "origin":[
           312.7900085449219,
           230.29998779296875
          ],
          "bbox":[
           312.7900085449219,
           219.93382263183594,
           559.2931518554688,
           233.41038513183594
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         312.7900085449219,
         219.93382263183594,
         559.2931518554688,
         233.41038513183594
        ]
       },
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"fadffdsdfgasddghFHFGHASFGDFFADSFD ",
          "origin":[
           312.7900085449219,
           244.70001220703125
          ],
          "bbox":[
           312.7900085449219,
           234.33384704589844,
           511.0224914550781,
           247.81040954589844
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         312.7900085449219,
         234.33384704589844,
         511.0224914550781,
         247.81040954589844
        ]
       }
      ]
     },
     {
      "number":6,
      "type":0,
      "bbox":[
       312.7900085449219,
       256.9237976074219,
       567.4398803710938,
       270.4003601074219
      ],
      "lines":[
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"Tststtsttsstssssstssssssssttss                                                  ",
          "origin":[
           312.7900085449219,
           267.28997802734375
          ],
          "bbox":[
           312.7900085449219,
           256.9237976074219,
           567.4398803710938,
           270.4003601074219
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         312.7900085449219,
         256.9237976074219,
         567.4398803710938,
         270.4003601074219
        ]
       }
      ]
     },
     {
      "number":7,
      "type":0,
      "bbox":[
       72.02400207519531,
       271.4438171386719,
       232.8125,
       284.9203796386719
      ],
      "lines":[
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"tsssssssssssssssssssssssssssss ",
          "origin":[
           72.02400207519531,
           281.80999755859375
          ],
          "bbox":[
           72.02400207519531,
           271.4438171386719,
           232.8125,
           284.9203796386719
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         72.02400207519531,
         271.4438171386719,
         232.8125,
         284.9203796386719
        ]
       }
      ]
     },
     {
      "number":8,
      "type":0,
      "bbox":[
       72.02400207519531,
       293.8838195800781,
       519.0430908203125,
       321.8803405761719
      ],
      "lines":[
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"HftTFADTYafdyttFADYfaddyfadyFGDYASFDUVASSHDVQHSDGVAHSVDDHSVDASVVKHASDFHA",
          "origin":[
           72.02400207519531,
           304.25
          ],
          "bbox":[
           72.02400207519531,
           293.8838195800781,
           519.0430908203125,
           307.3603820800781
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         72.02400207519531,
         293.8838195800781,
         519.0430908203125,
         307.3603820800781
        ]
       },
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"VSDHVASJDASK;LDKA;D  ",
          "origin":[
           72.02400207519531,
           318.76995849609375
          ],
          "bbox":[
           72.02400207519531,
           308.4037780761719,
           192.61248779296875,
           321.8803405761719
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         72.02400207519531,
         308.4037780761719,
         192.61248779296875,
         321.8803405761719
        ]
       }
      ]
     },
     {
      "number":9,
      "type":0,
      "bbox":[
       72.02400207519531,
       330.84381103515625,
       216.7324981689453,
       344.32037353515625
      ],
      "lines":[
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"SDAHSDAHDSLKJAS;LDKA\u2019SD ",
          "origin":[
           72.02400207519531,
           341.2099914550781
          ],
          "bbox":[
           72.02400207519531,
           330.84381103515625,
           216.7324981689453,
           344.32037353515625
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         72.02400207519531,
         330.84381103515625,
         216.7324981689453,
         344.32037353515625
        ]
       }
      ]
     },
     {
      "number":10,
      "type":0,
      "bbox":[
       72.02400207519531,
       353.40380859375,
       74.2665023803711,
       366.88037109375
      ],
      "lines":[
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":" ",
          "origin":[
           72.02400207519531,
           363.7699890136719
          ],
          "bbox":[
           72.02400207519531,
           353.40380859375,
           74.2665023803711,
           366.88037109375
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         72.02400207519531,
         353.40380859375,
         74.2665023803711,
         366.88037109375
        ]
       }
      ]
     },
     {
      "number":11,
      "type":0,
      "bbox":[
       72.02400207519531,
       215.0138397216797,
       201.7324981689453,
       228.4904022216797
      ],
      "lines":[
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"Name ",
          "origin":[
           72.02400207519531,
           225.3800048828125
          ],
          "bbox":[
           72.02400207519531,
           215.0138397216797,
           103.0625,
           228.4904022216797
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         72.02400207519531,
         215.0138397216797,
         103.0625,
         228.4904022216797
        ]
       },
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"Id ",
          "origin":[
           190.49000549316406,
           225.3800048828125
          ],
          "bbox":[
           190.49000549316406,
           215.0138397216797,
           201.7324981689453,
           228.4904022216797
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         190.49000549316406,
         215.0138397216797,
         201.7324981689453,
         228.4904022216797
        ]
       }
      ]
     },
     {
      "number":12,
      "type":0,
      "bbox":[
       72.02400207519531,
       228.93382263183594,
       198.61248779296875,
       242.41038513183594
      ],
      "lines":[
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"X ",
          "origin":[
           72.02400207519531,
           239.29998779296875
          ],
          "bbox":[
           72.02400207519531,
           228.93382263183594,
           80.3864974975586,
           242.41038513183594
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         72.02400207519531,
         228.93382263183594,
         80.3864974975586,
         242.41038513183594
        ]
       },
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"0 ",
          "origin":[
           190.49000549316406,
           239.29998779296875
          ],
          "bbox":[
           190.49000549316406,
           228.93382263183594,
           198.61248779296875,
           242.41038513183594
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         190.49000549316406,
         228.93382263183594,
         198.61248779296875,
         242.41038513183594
        ]
       }
      ]
     },
     {
      "number":13,
      "type":0,
      "bbox":[
       72.02400207519531,
       242.8538055419922,
       198.61248779296875,
       256.3303527832031
      ],
      "lines":[
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"Y ",
          "origin":[
           72.02400207519531,
           253.219970703125
          ],
          "bbox":[
           72.02400207519531,
           242.8538055419922,
           80.14649963378906,
           256.3303527832031
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         72.02400207519531,
         242.8538055419922,
         80.14649963378906,
         256.3303527832031
        ]
       },
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"1 ",
          "origin":[
           190.49000549316406,
           253.219970703125
          ],
          "bbox":[
           190.49000549316406,
           242.8538055419922,
           198.61248779296875,
           256.3303527832031
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         190.49000549316406,
         242.8538055419922,
         198.61248779296875,
         256.3303527832031
        ]
       }
      ]
     },
     {
      "number":14,
      "type":0,
      "bbox":[
       72.02400207519531,
       256.8038024902344,
       198.61248779296875,
       270.2803649902344
      ],
      "lines":[
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"z ",
          "origin":[
           72.02400207519531,
           267.16998291015625
          ],
          "bbox":[
           72.02400207519531,
           256.8038024902344,
           79.06649780273438,
           270.2803649902344
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         72.02400207519531,
         256.8038024902344,
         79.06649780273438,
         270.2803649902344
        ]
       },
       {
        "spans":[
         {
          "size":11.039999961853027,
          "flags":4,
          "font":"Aptos",
          "color":0,
          "ascender":0.93896484375,
          "descender":-0.28173828125,
          "text":"2 ",
          "origin":[
           190.49000549316406,
           267.16998291015625
          ],
          "bbox":[
           190.49000549316406,
           256.8038024902344,
           198.61248779296875,
           270.2803649902344
          ]
         }
        ],
        "wmode":0,
        "dir":[
         1.0,
         0.0
        ],
        "bbox":[
         190.49000549316406,
         256.8038024902344,
         198.61248779296875,
         270.2803649902344
        ]
       }
      ]
     }
    ]
   }
json_to_txt(json, 'output.txt')

In [34]:
from io import BytesIO
from img2table.document import PDF


def replace_from_table(json_data,output_file,cordinate,table):
    indx=0
    with open(output_file, 'w') as f:
        for block in json_data['blocks']:
            for line in block['lines']:
                for span in line['spans']:
                    x1, y1, x2, y2 = span['bbox']

                    if(x1<=cordinate[0],cordinate[2]<=x2 and y1<=cordinate[1],cordinate[3]<=y2):
                        if(indx==0):
                            span['text'] = table[0]+"\n"+table[1]
                            indx+=2

                        else:
                            span['text'] = table[indx]
                            indx+=1

                    width = x2 - x1
                    height = y2 - y1
                    # Calculate text position based on bbox coordinates
                    text_x = x1
                    text_y = y1 + height / 2  # Considering vertical center alignment
                    f.write(f"{text_x}, {text_y}, {span['text']}\n")  # Write text with position to file

def tabulate_converter(json_data):
    # Convert JSON to a list of dictionaries
    if type(json_data) == list:
        table_data = [list(record.values()) for record in json_data]
        # Get headers from the first record
        headers = list(json_data[0].keys())

    elif type(json_data) == dict:
        table_data = list(json_data.values())

        # Get headers from the first record
        headers = list(json_data.keys())

    # Convert to Markdown table
    markdown_table = tabulate(table_data, headers, tablefmt="pipe")

    return markdown_table



def table_titles(tables_dict):
    titles = []
    for t in tables_dict:
        if tables_dict[t] != []:
            t_index=1
            for table in tables_dict[t]:
                titles.append([(t+1, t_index),table.title])
                t_index+=1

    return titles



def table_to_json_based_xlsx(xlsx_path, titles):
    xlsx_tables = []
    xlsx_file = pd.ExcelFile(xlsx_path)  # Replace with your file path
    sheet_names = xlsx_file.sheet_names

    for sheet_name in sheet_names:
        df = pd.read_excel(xlsx_file, sheet_name=sheet_name)

        match = re.search(r"Page (\d+) - Table (\d+)", sheet_name)
        if match:
            page_number, table_number = map(int, match.groups())
        else:
            page_number=''
            table_number=''

        if page_number!='' and table_number!='':
            title=[k[1] for k in titles if k[0]==(page_number, table_number)][0]

        else:
            title=''

        if df.empty:
            temp = [{}]
            for k in range(len(df.columns)):
                try:
                    temp[0][df.columns[k]] = ''
                except:
                    continue

            xlsx_tables.append({'info': (page_number, table_number), 'title': title, 'data': json.dumps(temp)})
        else:
            xlsx_tables.append({'info': (page_number, table_number), 'title': title, 'data': df.to_json(orient="records")})

    return xlsx_tables


def table_extractor(pdf_path,xlsx_path):
    pdf_from_path = PDF(src=pdf_path, pages=[0, 1])

    # Definition of PDF from bytes
    with open(pdf_path, 'rb') as f:
        pdf_bytes = f.read()
    pdf_from_bytes = PDF(src=pdf_bytes)

    # print(pdf_from_bytes)

    pdf_from_file_like = PDF(src=BytesIO(pdf_bytes))

    # print("="*100)
    print(pdf_from_file_like)

    extracted_tables  = pdf_from_file_like.extract_tables()

    titles = table_titles(extracted_tables)
        
    tables=table_to_json_based_xlsx(xlsx_path, titles)

    return tables




pdf_path = "Test2.pdf"
xlsx_path = 'temp/tables.xlsx'
print(table_extractor(pdf_path,xlsx_path ))


PDF(src=<_io.BytesIO object at 0x0000028DE3C02520>, pages=None, detect_rotation=False, pdf_text_extraction=True, _rotated=False, _images=None)


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [33]:
!pip install openpyxl

In [21]:
def tabulate_converter(json_data):
    # Convert JSON to a list of dictionaries
    if type(json_data) == list:
        table_data = [list(record.values()) for record in json_data]
        # Get headers from the first record
        headers = list(json_data[0].keys())

    elif type(json_data) == dict:
        table_data = list(json_data.values())

        # Get headers from the first record
        headers = list(json_data.keys())

    # Convert to Markdown table
    markdown_table = tabulate(table_data, headers, tablefmt="pipe")

    return markdown_table



In [30]:
extracted_data = table_extractor(pdf_path)

if extracted_data:
    print(extracted_data)
    # Convert to JSON
    json_data = {header: data for header, data in extracted_data.items()}
    print(json_data)
else:
    print("No tables extracted from the PDF.")

PDF(src=<_io.BytesIO object at 0x0000028DE3BB9350>, pages=None, detect_rotation=False, pdf_text_extraction=True, _rotated=False, _images=None)
{0: [ExtractedTable(title=Kjlhsdhaslkjhkjdshfgjkbsvjbasjcbcbaknkscbkjbadckjjbasdckjbscjlbakcjbkkjascbkadbckjjbsackj
bsckjbassckjbasckjbasckkjbasckjbdackjjba
sckjjbckajjsbckjabsckjbdackjbadcbadkjbckajbdckbeakjbcawkjcjbassjjccbakjsdbckjabcskjjeab
khbasckjbasckjjbascljbascljcbasljcbaljsjcbljasbclajsbclajsbcljasbcaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaacabsk, bbox=(184, 596, 842, 751),shape=(4, 2))]}
{0: [ExtractedTable(title=Kjlhsdhaslkjhkjdshfgjkbsvjbasjcbcbaknkscbkjbadckjjbasdckjbscjlbakcjbkkjascbkadbckjjbsackj
bsckjbassckjbasckjbasckkjbasckjbdackjjba
sckjjbckajjsbckjabsckjbdackjbadcbadkjbckajbdckbeakjbcawkjcjbassjjccbakjsdbckjabcskjjeab
khbasckjbasckjjbascljbascljcbasljcbaljsjcbljasbclajsbclajsbcljasbcaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaacabsk, bbox=(184, 596, 842, 751),shape=(4, 2))]}


In [22]:
print(tabulate_converter({"name":["suren","sss"],"age":[12,34]}))

| name   | age   |
|:-------|:------|
| suren  | sss   |
| 12     | 34    |
